In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"
train_df = pd.read_csv(train_file_path,sep='\t',header=None,names=['label','message'])
test_df = pd.read_csv(test_file_path,sep='\t',header=None,names=['label','message'])


In [ ]:
print(f"Train shape: {train_df.shape}, Test shape: {test_df.shape}")

In [ ]:
# Encode labels: ham=0, spam = 1
train_df['label_num'] = train_df.label.map({'ham': 0, 'spam': 1})
test_df['label_num'] = test_df.label.map({'ham': 0, 'spam': 1})


In [ ]:
# Tokenize the text
max_vocab_size = 1000
tokenizer = keras.preprocessing.text.Tokenizer(
    num_words=max_vocab_size,
    oov_token='<OOV>'
    )
tokenizer.fit_on_texts(train_df['message'])


In [ ]:
print(train_df.columns)


In [ ]:

X_train = tokenizer.texts_to_sequences(train_df['message'])
X_test = tokenizer.texts_to_sequences(test_df['message'])

maxlen = 100
X_train = keras.preprocessing.sequence.pad_sequences(X_train,maxlen=maxlen,padding='post')
X_test = keras.preprocessing.sequence.pad_sequences(X_test,maxlen=maxlen,padding='post')

y_train = train_df['label_num'].values
y_test = test_df['label_num'].values

In [ ]:
# build model
model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_vocab_size,output_dim=16,input_length=maxlen),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
model.fit(X_train,y_train,epochs=1000,validation_data=(X_test,y_test),verbose=2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  seq = tokenizer.texts_to_sequences([pred_text])
  padded = keras.preprocessing.sequence.pad_sequences(seq,maxlen=maxlen,padding='post')
  pred = model.predict(padded)[0][0]
  label = 'spam' if pred>0.5 else 'ham'
  return [float(pred),label]

pred_text = "i dont want to go. can we try it a different day? available sat"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    # print(prediction)
    if prediction[1] != ans:
      passed = False


  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
